In [1]:
using Flux, MLDatasets, Statistics
using Flux: onehotbatch, onecold, crossentropy


In [2]:
model = Chain(
    Conv((3, 3), 1=>24, relu),
    x -> maxpool(x, (4,4)),

    x -> reshape(x, :, size(x, 4)),
    Dense(864, 32, relu),
    Dense(32, 10),

    softmax,
)


Chain(
  Conv((3, 3), 1 => 24, relu),          # 240 parameters
  var"#1#3"(),
  var"#2#4"(),
  Dense(864 => 32, relu),               # 27_680 parameters
  Dense(32 => 10),                      # 330 parameters
  NNlib.softmax,
)                   # Total: 6 arrays, 28_250 parameters, 110.953 KiB.

In [3]:
train_set = MNIST(split=:train, dir="./train")[:]
test_set  = MNIST(split=:test, dir="./test")[:]

(features = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; … ;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0;;; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], targets = [7, 2, 1, 0, 4, 1, 4, 9, 5, 9  …  7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [4]:
using Flux: Data.DataLoader

x_train = Flux.unsqueeze(train_set.features, 3)
x_test = Flux.unsqueeze(test_set.features, 3)

y_train = onehotbatch(train_set.targets, 0:9)
y_test = onehotbatch(test_set.targets, 0:9)

train_data = DataLoader((x_train, y_train); batchsize=64)
test_data = DataLoader((x_test, y_test); batchsize=64)

157-element DataLoader(::Tuple{Array{Float32, 4}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}, batchsize=64)
  with first element:
  (28×28×1×64 Array{Float32, 4}, 10×64 OneHotMatrix(::Vector{UInt32}) with eltype Bool,)

In [9]:
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

opt = ADAM(0.0075)

Adam(0.0075, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [11]:
using BenchmarkTools: @btime

number_epochs = 10
@btime for i in 1:number_epochs 
    Flux.train!(loss, Flux.params(model), train_data, opt)
    println("Epoch ", i, ". loss: ", loss(train_data.data[1], train_data.data[2])) 

end

Epoch 1. loss: 0.022134477
Epoch 2. loss: 0.040265255
Epoch 3. loss: 0.026367456
Epoch 4. loss: 0.02626853
Epoch 5. loss: 0.033919554
Epoch 6. loss: 0.023153849
Epoch 7. loss: 0.01687043
Epoch 8. loss: 0.02531087
Epoch 9. loss: 0.01582326
Epoch 10. loss: 0.028086687
Epoch 1. loss: 0.021297673
Epoch 2. loss: 0.033040334
Epoch 3. loss: 0.028732564
Epoch 4. loss: 0.016133768
Epoch 5. loss: 0.021547753
Epoch 6. loss: 0.01394144
Epoch 7. loss: 0.027098317
Epoch 8. loss: 0.032601345
Epoch 9. loss: 0.021819625
Epoch 10. loss: 0.021185588
Epoch 1. loss: 0.024594635
Epoch 2. loss: 0.028735299
Epoch 3. loss: 0.009092926
Epoch 4. loss: 0.03491487
Epoch 5. loss: 0.029173177
Epoch 6. loss: 0.017001465
Epoch 7. loss: 0.027858188
Epoch 8. loss: 0.031873193
Epoch 9. loss: 0.026064567
Epoch 10. loss: 0.028340254
Epoch 1. loss: 0.02013817
Epoch 2. loss: 0.046363708
Epoch 3. loss: 0.02701385
Epoch 4. loss: 0.014460394
Epoch 5. loss: 0.029796211
Epoch 6. loss: 0.013458872
Epoch 7. loss: 0.02554107
Epoch 8

In [7]:
acc = Vector{Float64}()

for data in test_data
    push!(acc, accuracy(data[1], data[2]))
        
end

In [8]:
println("Models accuracy:", mean!([1.], acc))

Models accuracy:[0.9811902866242038]
